In [ ]:
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
import pandas as pd
import json
import time

def scrape_comax_links():
    driver = uc.Chrome(version_main=135, headless=True, use_subprocess=True)

    base_url = "https://comax.uz"
    listing_url = base_url + "/ru/2-vse-tovary"
    product_links = []

    try:
        for page in range(1, 54):  # Pages 1 to 53
            url = f"{listing_url}?page={page}"
            print(f"[DEBUG] Scraping page {page} → {url}")
            driver.get(url)
            time.sleep(3)

            soup = BeautifulSoup(driver.page_source, "html.parser")
            json_ld_blocks = soup.find_all("script", type="application/ld+json")

            for block in json_ld_blocks:
                if block.string and "itemListElement" in block.string:
                    try:
                        data = json.loads(block.string)
                        for item in data.get("itemListElement", []):
                            name = item.get("name")
                            url = item.get("url")
                            if name and url:
                                product_links.append({"name": name, "product_url": url})
                        print(f"[DEBUG] Page {page}: Found {len(data['itemListElement'])} products")
                        break
                    except Exception as e:
                        print(f"[JSON-LD parse error on page {page}] {e}")
    except Exception as e:
        print(f"[comax fail] {e}")
    finally:
        driver.quit()

    print(f"[DEBUG] Total products collected: {len(product_links)}")
    return product_links


In [ ]:
products = scrape_comax_links()

In [ ]:
print(products)

In [ ]:
products = pd.DataFrame(products)

In [ ]:
products.to_pickle("products.pkl")